# Manipulación de datos (preprocesamiento)


In [ ]:
# Libraries to handle data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.datasets import load_wine
df_wine = load_wine(as_frame=True).frame

**Ejercicio 1.** Visualiza la primera fila del dataset.

**Ejercicio 2.** Muestra información general como el número de filas, el número de columnas, el nombre de las columnas y el tipo de dato de cada columna

**Ejercicio 3.** En la columna target, el número 0 corresponde a la etiqueta 'clase1', el 1 a 'clase2' y el 2 a 'clase3'. Modifica el dataset para que aparezcan los nombres en vez de los números.

In [ ]:
# Mapeo de los números a las etiquetas nominativas
mapeo_clases = {0: 'clase1', 1: 'clase2', 2: 'clase3'}

# Modificar la columna target
df_wine['target'] = df_wine['target'].map(mapeo_clases)

# Mostrar las primeras filas del DataFrame modificado para verificar el cambio
df_wine.head()


**Ejercicio 4.** Elimina la columna flavanoids y en su lugar crea una nueva que se llame graduacion_flavanoids. Esta nueva columna deberá tener como valores alto, medio y bajo dependiendo del valor de la columna flavanoids.

In [ ]:
# Definir las condiciones para la nueva columna graduacion_flavanoids
def clasificar_flavanoids(valor):
    if valor > df_wine['flavanoids'].quantile(0.66):
        return 'alto'
    elif valor > df_wine['flavanoids'].quantile(0.33):
        return 'medio'
    else:
        return 'bajo'

# Crear la nueva columna graduacion_flavanoids
df_wine['graduacion_flavanoids'] = df_wine['flavanoids'].apply(clasificar_flavanoids)

# Eliminar la columna flavanoids
df_wine.drop(columns=['flavanoids'], inplace=True)

# Mostrar las primeras filas del DataFrame modificado para verificar los cambios
df_wine.head()


**Ejercicio 5.** Comprueba el número de ocurrencias por cada una de las clases que queremos predecir.

In [ ]:
# Contar el número de ocurrencias de cada una de las clases en la columna target
ocurrencias_clases = df_wine['target'].value_counts()

# Mostrar los resultados
ocurrencias_clases


**Ejercicio 6.** Balancea el dataset teniendo en cuenta la variable target. Deberá haber el mismo número de ocurrencias de cada uno de los valores de target, para ello deberás añadir nuevas a la que menos tenga y quitar a la que más tenga.

In [ ]:
from sklearn.utils import resample

# Contar el número de ocurrencias de cada clase
ocurrencias_clases = df_wine['target'].value_counts()
max_ocurrencias = ocurrencias_clases.max()

# Crear un DataFrame vacío para almacenar las clases balanceadas
df_wine_balanceado = pd.DataFrame()

# Balancear cada clase
for clase in ocurrencias_clases.index:
    clase_df = df_wine[df_wine['target'] == clase]
    if len(clase_df) < max_ocurrencias:
        # Aumentar la clase minoritaria
        clase_df_balanceada = resample(clase_df, replace=True, n_samples=max_ocurrencias, random_state=42)
    else:
        # Reducir la clase mayoritaria
        clase_df_balanceada = clase_df.sample(n=max_ocurrencias, random_state=42)

    # Añadir al DataFrame balanceado
    df_wine_balanceado = pd.concat([df_wine_balanceado, clase_df_balanceada])

# Mostrar el número de ocurrencias de cada clase en el DataFrame balanceado
df_wine_balanceado['target'].value_counts()


**Ejercicio 7.** Realiza una codificación one-hot encoding de la variable graduacion_flavanoids

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Inicializar OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Realizar la codificación one-hot encoding de la variable graduacion_flavanoids
graduacion_encoded = encoder.fit_transform(df_wine_balanceado[['graduacion_flavanoids']])

# Crear un DataFrame con las nuevas columnas codificadas
graduacion_encoded_df = pd.DataFrame(graduacion_encoded, columns=encoder.get_feature_names_out(['graduacion_flavanoids']))

# Combinar el DataFrame original sin la columna graduacion_flavanoids con el DataFrame codificado
df_wine_one_hot = pd.concat([df_wine_balanceado.drop(columns=['graduacion_flavanoids']), graduacion_encoded_df], axis=1)

# Mostrar las primeras filas del DataFrame para verificar el resultado
df_wine_one_hot.head()


**Ejercicio 8.** Aplica un filtro para eliminar aquellas ocurrencias en las que 'bajo', 'medio' y 'alto' es igual a 0 o 'bajo', 'medio' y 'alto' es igual a 1. **OJO. no debería haber ninguna.**

In [ ]:
# Verificar que no haya ocurrencias donde todas las columnas one-hot encoding sean 0 o 1
condicion_a_eliminar = (
    (df_wine_one_hot['graduacion_flavanoids_bajo'] == 0) &
    (df_wine_one_hot['graduacion_flavanoids_medio'] == 0) &
    (df_wine_one_hot['graduacion_flavanoids_alto'] == 0)
) | (
    (df_wine_one_hot['graduacion_flavanoids_bajo'] == 1) &
    (df_wine_one_hot['graduacion_flavanoids_medio'] == 1) &
    (df_wine_one_hot['graduacion_flavanoids_alto'] == 1)
)

# Aplicar el filtro para eliminar las ocurrencias no deseadas
df_wine_filtrado = df_wine_one_hot[~condicion_a_eliminar]

# Mostrar las primeras filas del DataFrame filtrado para verificar el resultado
df_wine_filtrado.head()

# Verificar si se han eliminado ocurrencias
eliminadas = df_wine_one_hot.shape[0] - df_wine_filtrado.shape[0]
print(f"Número de ocurrencias eliminadas: {eliminadas}")


**Ejercicio 9.** Muestra qué variables categóricas y numéricas hay.

In [ ]:
# Identificar las variables categóricas y numéricas
variables_categoricas = df_wine_filtrado.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
variables_numericas = df_wine_filtrado.select_dtypes(include=['number']).columns.tolist()

# Mostrar las variables categóricas y numéricas
print("Variables categóricas:", variables_categoricas)
print("Variables numéricas:", variables_numericas)


**Ejercicio 10.** Crea una función que convierta todas las variables categóricas en numéricas aplicando un mapeo. Dicha función devolverá el dataset modificado. Utiliza LabelEncoder.

In [ ]:
from sklearn.preprocessing import LabelEncoder

def convertir_categoricas_a_numericas(df):
    # Crear una copia del DataFrame para no modificar el original
    df_modificado = df.copy()

    # Identificar las variables categóricas
    variables_categoricas = df_modificado.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

    # Inicializar el LabelEncoder
    label_encoder = LabelEncoder()

    # Convertir cada variable categórica en numérica
    for columna in variables_categoricas:
        df_modificado[columna] = label_encoder.fit_transform(df_modificado[columna])

    return df_modificado

# Aplicar la función al DataFrame filtrado
df_wine_numerico = convertir_categoricas_a_numericas(df_wine_filtrado)

# Mostrar las primeras filas del DataFrame modificado
df_wine_numerico.head()


**Ejercicio 11.** Normaliza el dataset al completo excepto la columna target.

**Ejercicio 12.** Divide el dataset en tres grupos. El primer grupo será de entrenamiento y tendrá el 80% de los datos. El segundo grupo será de validación y tendrá el 10% de los datos. El tercer grupo será de test y tendrá el 10% de los datos.